In [1]:
# Ejercicio 14

import numpy as np

In [3]:
P = np.array([[1/4, 1/2, 1/2, 1/2],
              [1/4, 1/2,   0,   0],
              [1/4,   0, 1/2,   0],
              [1/4,   0,   0, 1/2]])
P

array([[0.25, 0.5 , 0.5 , 0.5 ],
       [0.25, 0.5 , 0.  , 0.  ],
       [0.25, 0.  , 0.5 , 0.  ],
       [0.25, 0.  , 0.  , 0.5 ]])

In [8]:
Plim = P
for i in range(30):
    Plim = Plim @ P
Plim

array([[0.4, 0.4, 0.4, 0.4],
       [0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2]])

In [124]:
# Ejercicio 15: Caminata aleatoria

def crearMatrizTransicion(p=0.5, n=100):
    # Creo matriz de transicion como matriz con diagonales que envuelven a la principal
    # con probabilidad p en la inferior (volver un paso atrás hacia el bar)
    # y 1-p en la superior (avanzar 1 paso hacia la casa)
    P = np.eye(n + 1, k=-1) * p + np.eye(n + 1, k=1) * (1 - p)
    # Necesito agregar los valores terminales en la primera y ultima columna/fila
    P[:, 0] = 0 # Borro primera columna
    P[:, n] = 0 # Borro ultima columna
    P[0, 0] = 1
    P[n, n] = 1

    return P

In [125]:
p = 0.75 # Probabilidad de dar un paso hacia su casa
n = 5
crearMatrizTransicion(p, n)

array([[1.  , 0.25, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.25, 0.  , 0.  , 0.  ],
       [0.  , 0.75, 0.  , 0.25, 0.  , 0.  ],
       [0.  , 0.  , 0.75, 0.  , 0.25, 0.  ],
       [0.  , 0.  , 0.  , 0.75, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.75, 1.  ]])

In [221]:
import matplotlib.pyplot as plt

In [127]:
# b)
p = 0.5 # Probabilidad de dar un paso hacia su casa
n = 20
A = crearMatrizTransicion(p, n)
A[:4, :4]

array([[1. , 0.5, 0. , 0. ],
       [0. , 0. , 0.5, 0. ],
       [0. , 0.5, 0. , 0.5],
       [0. , 0. , 0.5, 0. ]])

In [135]:
# Eigenvalues y eigenvectors
eigvals, eigvecs = np.linalg.eig(A)
eigvals.round(3)

array([ 1.   ,  1.   , -0.988, -0.951, -0.891, -0.809, -0.707, -0.588,
       -0.454, -0.309, -0.156,  0.156,  0.309,  0.988,  0.951,  0.891,
        0.809,  0.707,  0.454,  0.588,  0.   ])

In [263]:
#v0 = np.ones(n + 1) / (n + 1)
v0 = np.zeros(n + 1)
v0[n//2] = 1
v0

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])

In [264]:
v = v0
vs = [v]

In [307]:
for i in range(100):
    v = A @ v
    vs.append(v)
print(v.round(3))
# Plot animation of distribution of array vs using plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

fig = make_subplots(rows=1, cols=1, subplot_titles=("Distribución"))

fig.add_trace(
    go.Bar(x=np.arange(n + 1), y=v, name="Distribución"),
    row=1, col=1
)
fig.update_layout(height=600, width=800, title_text="Caminata aleatoria")
fig.show()




[0.453 0.    0.005 0.    0.009 0.    0.012 0.    0.014 0.    0.015 0.
 0.014 0.    0.012 0.    0.009 0.    0.005 0.    0.453]


In [306]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np

# Create your array of probability distributions at different time steps.
# Let's assume you have a list of arrays called `distribution_data`, where each array represents a probability distribution.

# Create a figure with subplots
fig = make_subplots(rows=1, cols=1)

# Set up the initial plot (assuming distribution_data[0] is the initial distribution)
x = np.arange(len(vs[0]))  # Assuming your data is discrete
trace = go.Bar(x=x, y=vs[0], name='Time Step 0')

# Add the trace to the subplot
fig.add_trace(trace, row=1, col=1)

# Define initial animation duration
animation_duration = 100  # 1 second

# Function to update the plot for each frame of the animation
def update(frame):
    # Create a new trace for the current distribution
    new_trace = go.Bar(x=x, y=vs[frame], name=f'Time Step {frame}')

    # Update the subplot with the new trace
    fig.data = [new_trace]

    # Update the title
    fig.update_layout(title=f'Time Step {frame}')

# Create the animation
num_frames = len(vs)

animation_frames = [go.Frame(data=[go.Bar(x=x, y=vs[frame], name=f'Time Step {frame}')],
                            name=f'Time Step {frame}')
                    for frame in range(num_frames)]

fig.frames = animation_frames

# Define the layout for your plot
layout = go.Layout(
    updatemenus=[
        {
            'buttons': [
                {
                    'args': [None, {'frame': {'duration': animation_duration, 'redraw': True}, 'fromcurrent': True}],
                    'label': 'Play',
                    'method': 'animate',
                },
                {
                    'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 0}}],
                    'label': 'Pause',
                    'method': 'animate',
                },
            ],
            'direction': 'left',
            'pad': {'r': 10, 't': 87},
            'showactive': False,
            'type': 'buttons',
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top',
        },
    ],
    sliders=[{
        'active': 0,
        'yanchor': 'top',
        'xanchor': 'left',
        'currentvalue': {
            'font': {'size': 20},
            'prefix': 'Time Step:',
            'visible': True,
            'xanchor': 'right',
        },
        'transition': {'duration': 300, 'easing': 'cubic-in-out'},
        'pad': {'b': 10, 't': 50},
        'len': 0.9,
        'x': 0.1,
        'y': 0,
        'steps': [{'args': [[f'Time Step {frame}'], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
                   'label': f'Time Step {frame}',
                   'method': 'animate'} for frame in range(num_frames)],
    }],
)

fig.update_layout(layout)

# Show the animation
fig.show()
